# Pre-modelo de clasificación basado en texto

Muy seguramente no bastará con la descripción del videojuego para poder predecir su valor monetario. Entrenaremos un modelo pretendiendo que si y luego extraeremos parte de este para agregar features en un clasificador más robusto.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Cargando el dataset

In [2]:
df_train = pd.read_pickle('train.pickle')

X_train, X_eval, y_train, y_eval = train_test_split(df_train, df_train['rating'], test_size=0.3, random_state=0, stratify=df_train['rating'])

In [3]:
X_train = X_train.drop(columns=['name','release_date','english','developer','publisher','platforms','required_age','categories','genres','tags','achievements','average_playtime','price','estimated_sells'])
X_eval = X_eval.drop(columns=['name','release_date','english','developer','publisher','platforms','required_age','categories','genres','tags','achievements','average_playtime','price','estimated_sells'])

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5516 entries, 3997 to 2515
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   short_description  5516 non-null   object  
 1   rating             5516 non-null   category
dtypes: category(1), object(1)
memory usage: 91.6+ KB


In [5]:
X_train['rating'] = X_train['rating'].astype(str)
X_eval['rating'] = X_eval['rating'].astype(str)

In [6]:
from datasets import Dataset

data_train = Dataset.from_pandas(X_train)
data_eval = Dataset.from_pandas(X_eval)

In [7]:
print(data_train[0])
data_train

{'short_description': 'Play as the hunter girl Azel and battle monsters with equipment made of their corpses while searching for the Black Demon Beast that branded her.', 'rating': 'Very Positive', '__index_level_0__': 3997}


Dataset({
    features: ['short_description', 'rating', '__index_level_0__'],
    num_rows: 5516
})

In [8]:
from datasets import ClassLabel

data_train = data_train.cast_column("rating",ClassLabel(num_classes=5, names=['Negative', 'Mixed', 'Mostly Positive', 'Positive', 'Very Positive']))
data_eval = data_eval.cast_column("rating",ClassLabel(num_classes=5, names=['Negative', 'Mixed', 'Mostly Positive', 'Positive', 'Very Positive']))

Casting the dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
data_train.features

{'short_description': Value(dtype='string', id=None),
 'rating': ClassLabel(names=['Negative', 'Mixed', 'Mostly Positive', 'Positive', 'Very Positive'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

Queremos hacer finetuning a BETO para la tarea de predecir los ratings, lo cual corresponde a una tarea de clasificación. Seguiremos partes de [este tutorial](https://huggingface.co/docs/transformers/tasks/sequence_classification).

## Cargando tokenizer y modelos pre-entrenados

In [17]:
from transformers import AutoTokenizer

model_id = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/camilo/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

loading file vocab.txt from cache at /home/camilo/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /home/camilo/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/camilo/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /home/camilo/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout":

In [18]:
def preprocess_function(examples):
    return tokenizer(examples["short_description"], truncation=True, padding=True)

In [19]:
data_train = data_train.map(preprocess_function, batched=True)
data_eval = data_eval.map(preprocess_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=5)

loading configuration file config.json from cache at /home/camilo/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "distilbert-movie-descriptions-rating",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset = data_train,
    eval_dataset = data_eval,
    tokenizer = tokenizer,
    data_collator = data_collator,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Fine-tunning

In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, short_description, rating. If __index_level_0__, short_description, rating are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5516
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1725
  Number of trainable parameters = 108314117


  0%|          | 0/1725 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
trainer.save_model("distilbert-movie-descriptions-rating")